# ScrudHandler
A base handler for a SCRUD-compatible API

> in a proped server config, we would expose a `register` method at the module level, or particpate in a `jupyter_config`

In [8]:
from notebook.base.handlers import IPythonHandler
from tornado import escape, web
import asyncio, jsonschema

In [ ]:
CTX_MIME = "application/ld+json"
CTX_REL = "http://www.w3.org/ns/json-ld#context"
CTX_DOC = "@context"

In [6]:
SCH_MIME = "application/json"
SCH_REL = "describedBy"
SCH_DOC = "$schema"

In [7]:
class ScrudHandler(IPythonHandler):
    def initialize(self, url_base, *args, **kwargs):
        super().initialize(*args, **kwargs)
        self._url_base = url_base

    @web.authenticated
    async def get(self, doc=None):
        self.make_headers(doc)
        if doc == "@context":
            self.finish(escape.json_encode(await self.context()))
        elif doc == "$schema":
            self.finish(escape.json_encode(await self.schema()))
        else:
            schema, context, frame, value = await asyncio.gather(
                self.schema(), self.context(), self.frame(), self.value()
            )

            try:
                jsonschema.validate(value, schema)
                self.finish(value)
            except Exception as err:
                self.finish({"errors": [f"{err}"]})
    
    def make_headers(self, doc):
        if doc == "@context":
            self.set_header('Content-Type', CTX_MIME)
        elif doc == "$schema":
            self.set_header('Content-Type', SCH_MIME)
        else:
            self.add_header(
                "Link", 
                f'''<{self._url_base}{CTX_DOC}>; rel="{CTX_REL}"; type="{CTX_MIME}"'''
            )
            self.add_header(
                "Link", 
                f'''<{self._url_base}{SCH_DOC}>; rel="{SCH_REL}"; type="{SCH_MIME}"'''
            )
    
    @web.authenticated
    async def head(self, doc=None):
        self.make_headers(doc)
        
    # SCRUD-specific values and documents
    
    async def value(self):
        return None
    
    async def schema(self):
        return {}
    
    async def frame(self):
        return {}
    
    async def context(self):
        return {}